# Lab 6 - Query feature extraction with OpenAI "Function Calling"

In [6]:
import os
import openai
import json
import re
import pandas as pd
from tqdm.notebook import tqdm

#Caching function responses lets us iterate without re-calling expensive operations
import functools

#Don't include keys like this, use ENV vars!
with open('config.json') as fd:
    conf = json.loads(fd.read())
    gpt = openai.OpenAI(api_key=conf["openai_key"])

ModuleNotFoundError: No module named 'openai'

In [ ]:
@functools.cache
def get_gpt_completion(context):
    messages= [{"role": "user", "content": context}]
    functions= [
        {
            "name": "extract_property_search_values",
            "description": "For a property search system, we need to extract intent based values from a user query.",
            "parameters": { 
                "type": "object",
                "properties": { #Follows JSON Schema conventions: https://json-schema.org/understanding-json-schema/
                  "bedrooms": {
                    "type": "integer",
                    "description": "The number of bedrooms"
                  },
                  "square_footage": {
                    "type": "integer",
                    "description": "Total square footage"
                  },
                  "features": {
                    "type": "string",
                    "description": "Any additional features or architectural styles"
                  },
                  "non_property_related": {
                    "type": "boolean",
                    "description": "Flag to indicate if the query is not related to property search"
                  }
                },
                "required": ["bedrooms","square_footage","features","non_property_related"]
            }
        }
    ]
    
    response = gpt.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    
    return response

In [2]:
def extract_search_values(querystring):
    gpt_res = get_gpt_completion(querystring)

    if gpt_res:
        #print('OPENAI RESPONSE:',gpt_res)
        response_message=gpt_res.choices[0].message
    
        # Check if the model wants to call a function
        if response_message.function_call:
        
            # Call the function. The JSON response may not always be valid so make sure to handle errors
            function_name = response_message.function_call.name
            if(function_name=="extract_property_search_values"):
                function_args = json.loads(response_message.function_call.arguments)
                return function_args

In [3]:
extract_search_values("Nice colonial house with 3 or more bedrooms and a pool and white picket fence")

NameError: name 'get_gpt_completion' is not defined

In [ ]:
extract_search_values("Who is Mr. T?")

NameError: name 'get_gpt_completion' is not defined